In [1]:
%matplotlib inline

## Intro

Este tutorial é baseado no tutorial [Aprendendo representações de Phrases utilizando um codificador-decodificador com RNN](https://github.com/bentrevett/pytorch-seq2seq/blob/master/2%20-%20Learning%20Phrase%20Representations%20using%20RNN%20Encoder-Decoder%20for%20Statistical%20Machine%20Translation.ipynb)

### Arquitetura de um codificador-decodificador

Vamos lembrar da visão geral de um codificador-decodificador:

![alt text](seq2seq1.png "Title")

Nós usamos o codificador (em verde) na sequência de entrada para gerar um vetor de contexto `z` (em vermelho).
Esse vetor é então utilizado em um decodificador (em azul) e uma camada linear (em roxo) para gerar a sequência de saída.

Neste modelo, estamos usando um modelo de múltiplas camadas implementando uma memória de curto e longo prazo (`LSTM`):

![alt text](seq2seq4.png "Title")

Um dos problemas deste modelo, é que o decodificador está tentando colocar muita informação nos estados intermediários do nosso modelo.  No momento da decodificação, o estado intermediário deverá conter informação sobre toda a sequência de entrada codificada até o momento bem como todos os tokens decodificados até então. Isto exige muita memória. Seria interessante amenizar o processo de compressão para que possamos ter um modelo melhor!

Para isso, usaremos uma GRU... mais detalhes

## Dados

Inicialmente, vamos importar algumas das bibliotecas necessárias para manipular os nossos dados:

In [8]:

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import os
import time

Em seguida, vamos usar a mesma `SEED` para garantir que os nossos resultados sejam reproduzíveis/determinísticos

In [9]:
SEED = 1

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Por fim, vamos utilizar modelos em alemão e inglês. 

No seu ambiente conda, execute a linha de comando abaixo para baixar os modelos:


```bash
python -m spacy download en
python -m spacy download de
```


Em caso de sucesso, você obterá uma mensagem similar a:


```bash
    You can now load the model via spacy.load('pt')
```

In [19]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

Ao processar os textos de entrada, vamos utilizar uma técnica chamada de tokenização

TODO: mais detalhes

In [21]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Também vamos definir dois tokens especiais para indicar o início de uma frase (`sos`) e o final de uma frase (`eos`). 

In [22]:
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

Finalmente, vamos carregar nossos dados

In [23]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

Vamos checar uma frase de entrada e saída para garantir que nossos dados estão corretos:

In [25]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(vars(train_data.examples[0]))

{   'src': [   'zwei',
               'junge',
               'weiße',
               'männer',
               'sind',
               'im',
               'freien',
               'in',
               'der',
               'nähe',
               'vieler',
               'büsche',
               '.'],
    'trg': [   'two',
               'young',
               ',',
               'white',
               'males',
               'are',
               'outside',
               'near',
               'many',
               'bushes',
               '.']}


Por fim, vamos construir o nosso vocabulário convertendo todos os tokens que aparecem menos de duas vezes em termos desconhecidos (`<unk>`)

In [26]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

Por fim, vamos dividir os nossos dados em dados de treinamento, validação e testes. 

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device)

## Construindo o nosso modelo

### Encoder




In [28]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

### Decoder

In [29]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.out = nn.Linear(emb_dim + hid_dim*2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim=2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #sent len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim=1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

### Seq2Seq 

In [30]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, max_len):
            
            output, hidden = self.decoder(input, hidden, context)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)

        return outputs

In [31]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [32]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,219,781 trainable parameters


In [33]:
optimizer = optim.Adam(model.parameters())

In [34]:
pad_idx = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [35]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [37]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [38]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Training this model without a CUDA takes 1h and 20min!!

In [39]:
N_EPOCHS = 10
CLIP = 1
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'tut2_model.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 10m 32s
	Train Loss: 4.633 | Train PPL: 102.818
	 Val. Loss: 4.308 |  Val. PPL:  74.325
Epoch: 02 | Time: 9m 16s
	Train Loss: 3.617 | Train PPL:  37.217
	 Val. Loss: 3.841 |  Val. PPL:  46.550
Epoch: 03 | Time: 9m 16s
	Train Loss: 3.173 | Train PPL:  23.870
	 Val. Loss: 3.670 |  Val. PPL:  39.247
Epoch: 04 | Time: 9m 9s
	Train Loss: 2.868 | Train PPL:  17.603
	 Val. Loss: 3.610 |  Val. PPL:  36.958
Epoch: 05 | Time: 9m 5s
	Train Loss: 2.630 | Train PPL:  13.877
	 Val. Loss: 3.639 |  Val. PPL:  38.051
Epoch: 06 | Time: 9m 11s
	Train Loss: 2.439 | Train PPL:  11.463
	 Val. Loss: 3.600 |  Val. PPL:  36.598
Epoch: 07 | Time: 9m 11s
	Train Loss: 2.304 | Train PPL:  10.017
	 Val. Loss: 3.552 |  Val. PPL:  34.890
Epoch: 08 | Time: 9m 8s
	Train Loss: 2.176 | Train PPL:   8.812
	 Val. Loss: 3.628 |  Val. PPL:  37.635
Epoch: 09 | Time: 9m 14s
	Train Loss: 2.084 | Train PPL:   8.033
	 Val. Loss: 3.647 |  Val. PPL:  38.364
Epoch: 10 | Time: 9m 13s
	Train Loss: 2.000 | Train PPL: 

In [40]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.528 | Test PPL:  34.046 |
